In [1]:
%%capture
## compile PyRoss for this notebook
import os
owd = os.getcwd()
os.chdir('../../')
%run setup.py install
os.chdir(owd)

In [1]:
import numpy as np
import pyross

In [2]:
# Get individual contact matrices
CH, CW, CS, CO = pyross.contactMatrix.UK()

In [3]:
# Generate class with contact matrix for SIR model with UK contact structure
generator = pyross.contactMatrix.ContactMatrixFunction(CH, CW, CS, CO)

In [4]:
# get constant contact matrix function (this is plug-and-play for model.simulate)
C = generator.constant_contactMatrix()

In [5]:
# compare: Does the constant contact matrix function yield the sum of the individual contact matrices?
(CH + CW + CS + CO == C(123)).all()

True

## Create matrix for temporal intervention

In [12]:
times= [1, 2, 3] # temporal boundaries between different contact-behaviour
# prefactors for CW, CS, CO:
interventions = [[0.9,0.9,0.8], # before first time
                 [0.5,0.4,0.3], # between first and second time
                 [0.8,0.7,0.6], # between second and third time
                  [0.9,0.3,0.5]] # for times larger than third time

# generate corresponding contact matrix function
C = generator.interventions_temporal(times=times,interventions=interventions)

In [32]:
# Check: Does the contact matrix function give what it should?

# times at which to evaluate contact matrix function
test_times = [-0.1, 0.5, 1.1, 2.9, 5]
# for each time, we here enter the corresponding row from the "interventions" matrix by hand
test_indices = [0, 0, 1, 2, 3]
 
for i,t in enumerate(test_times):
    j = test_indices[i]
    lhs = CH + interventions[j][0]**2*CW + interventions[j][1]**2*CS + interventions[j][2]**2*CO
    rhs = C(t) 
    print(np.allclose(lhs, rhs))

True
True
True
True
True


## Create matrix for population-threshold driven intervention

In [33]:
# For the SIR model, we have 3*M population numbers (S,Ia,Is)
# We now consider M = 1
# (Possible extension: Include class R)

# thresholds for switching
thresholds = np.array([ [0,20,0],
                        [0,40,0],
                         [0, 100, 0]])
# interventions
interventions = [[0.5,0.2,0.3], # before first time
                 [0.2,0.1,0.1], # between first and second time
                 [0.4,0.5,0.2], # between second and third time
                  [0.7,0.1,0.1]] # for times larger than third time

# generate contact matrix function
C = generator.interventions_threshold(thresholds=thresholds,interventions=interventions)

# Note that this contact matrix function now takes 4 arguments!
# C == C(t, S, Ia, Is)

In [34]:
# Check: Does the contact matrix function give what it should?

test_populations = [ [ 10,10,10] ,
                      [0, 10, 10],
                       [10, 50,100],
                      [100,200,0]]

# The function is written so that for a "population vector" state = (S,Ia,Is),
# the intervention with the largest index i such that
# state[j] >= thresholds[index,j] for all j
# is chosen. 
# Put differently, we look for the first row (from above) in the matrix "thresholds" such that
# all population numbers exceed the threshold.

# for each time, we here enter the corresponding row from the "interventions" matrix by hand
test_indices = [0, 0, 2, 3]